In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import numpy as np
import s3fs
import io
import os
import gc
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
import pyspark.sql.functions as F
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess
import boto3
from botocore.config import Config 
from typing import Tuple,List
import sys
import socket
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import time 
import pandas as pd

In [2]:
MASTER_IP = "192.168.128.236"  # IP del Server Master
MINIO_BUCKET_NAME="satellite-data"
MINIO_ADDRESS="http://192.168.128.236:9000"
# Configurazione MinIO
MINIO_ENDPOINT = f"http://{MASTER_IP}:9000"
MINIO_ACCESS_KEY = "minioadmin"
MINIO_SECRET_KEY = "minioadmin"


## Setup Spark context 

In [3]:
# Trova automaticamente l'IP della TUA macchina (Client PC7)
# che è visibile al Master.
def get_local_ip(target_ip):
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
        try:
            # Non invia dati, serve solo a capire quale interfaccia di rete usare
            s.connect((target_ip, 80)) 
            return s.getsockname()[0]
        except Exception:
            return "127.0.0.1"

CLIENT_IP = get_local_ip(MASTER_IP)
print(f"📡 Il mio IP Client (Driver) è: {CLIENT_IP}")
print(f"🎯 Mi connetto al Master su: {MASTER_IP}")



# -------------------------------------------------------
# 2. Pulizia sessioni precedenti (Anti-Zombie)
# -------------------------------------------------------
try:
    if 'spark' in globals(): spark.stop()
    if SparkContext._active_spark_context: SparkContext._active_spark_context.stop()
    SparkContext._active_spark_context = None
except Exception:
    pass

# -------------------------------------------------------
# 3. Inizializzazione Spark Session
# -------------------------------------------------------
conf = SparkConf()
conf.setAppName("SatStream-Distributed-Client")
conf.setMaster(f"spark://{MASTER_IP}:7077")

# CONFIGURAZIONE RETE FONDAMENTALE
# Il Driver (questo PC) deve essere raggiungibile dai Worker remoti
conf.set("spark.driver.host", CLIENT_IP)        # <--- CORRETTO: Usa l'IP del Client, non del Master
conf.set("spark.driver.bindAddress", "0.0.0.0") # Ascolta su tutte le interfacce locali
conf.set("spark.driver.port", "20000")          # Porta fissa per firewall
conf.set("spark.blockManager.port", "20001")    # Porta fissa per dati

# Configurazione S3/MinIO
conf.set("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT)
conf.set("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY)
conf.set("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY)
conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

conf.set("spark.jars.packages", 
         "org.apache.hadoop:hadoop-aws:3.3.4,"
         "org.apache.hadoop:hadoop-common:3.3.4,"
         "com.amazonaws:aws-java-sdk-bundle:1.12.262")

# Timeout e Heartbeat (utili in reti distribuite)
conf.set("spark.network.timeout", "600s")
conf.set("spark.executor.heartbeatInterval", "60s")

print("🚀 Tentativo di connessione al Cluster...")

try:
    spark = SparkSession.builder.config(conf=conf).getOrCreate()
    
    print(f"\n✅ SUCCESS! Connesso al Cluster.")
    print(f"🔗 Master: {spark.sparkContext.master}")
    print(f" Web UI (Locale): http://{CLIENT_IP}:4040")
    print(f" Web UI (Master): http://{MASTER_IP}:8080")
    
    # Test veloce
    t=time.time()
    count = spark.range(100).count()
    t=time.time()-t
    print(f"🧪 Test di calcolo distribuito: {count} righe contate in {t} secondi.")
    
except Exception as e:
    print(f"\n❌ ERRORE CRITICO:\n{e}")
    print("\nSUGGERIMENTO: Controlla che la porta 7077 sul Master sia aperta e che 'spark.driver.host' sia raggiungibile dai Worker.")

📡 Il mio IP Client (Driver) è: 192.168.128.236
🎯 Mi connetto al Master su: 192.168.128.236
🚀 Tentativo di connessione al Cluster...


25/12/29 14:52:37 WARN Utils: Your hostname, PC7 resolves to a loopback address: 127.0.1.1; using 192.168.128.236 instead (on interface eno1)
25/12/29 14:52:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/amministratore/miniconda3/envs/spark_env/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/amministratore/.ivy2/cache
The jars for the packages stored in: /home/amministratore/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f99ce4d2-ab5f-4307-99ae-5fc657c5cf94;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.apache.hadoop#hadoop-common;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-protobuf_3_7;1.1.1 in central
	found org.apache.hadoop#hadoop-annotations;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-guava;1.1.1 in central
	found com.google.guava#guava;27.0-jre in central
	found com.google.guava#failureaccess;1.0 in central
	found com.google.guava#listenabl


✅ SUCCESS! Connesso al Cluster.
🔗 Master: spark://192.168.128.236:7077
 Web UI (Locale): http://192.168.128.236:4040
 Web UI (Master): http://192.168.128.236:8080


🧪 Test di calcolo distribuito: 100 righe contate in 6.986287832260132 secondi.


## Verifica dataset MinIO (boto3 + mc)

In [4]:
# 🔧 BYPASS S3 Spark → Usa mc + Python nativo (100% funziona)
print(" Lista patch via mc (conferma esistenza)")
result = subprocess.run([
    "docker", "exec", "minio", "mc", "ls", 
    "myminio/satellite-data/parquet/", "|", "wc", "-l"
], shell=True, capture_output=True, text=True)

print(result.stdout)
print(" mc conferma patch esistono!")

# 🔧 S3 client Python nativo (bypassa Spark S3A)
s3 = boto3.client(
    's3',
    endpoint_url=MINIO_ADDRESS,
    aws_access_key_id=MINIO_ACCESS_KEY,
    aws_secret_access_key=MINIO_SECRET_KEY
)

response = s3.list_objects_v2(Bucket=MINIO_BUCKET_NAME, Prefix='parquet/')
files = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.parquet')]
print(f" {len(files):,} patch .npz confermate via boto3!")

# Prime 10
print(" Prime 10:")
for f in files[:10]:
    size = next(obj['Size'] for obj in response['Contents'] if obj['Key'] == f)
    print(f"  {size/1024/1024:.1f}MB {f}")




 Lista patch via mc (conferma esistenza)

 mc conferma patch esistono!
 1,000 patch .npz confermate via boto3!
 Prime 10:
  6.0MB parquet/task_0_worker_9.parquet
  5.6MB parquet/task_1000_worker_2.parquet
  5.6MB parquet/task_1001_worker_9.parquet
  5.6MB parquet/task_1002_worker_9.parquet
  5.6MB parquet/task_1004_worker_3.parquet
  5.6MB parquet/task_1005_worker_2.parquet
  5.6MB parquet/task_1007_worker_9.parquet
  5.5MB parquet/task_1009_worker_9.parquet
  5.4MB parquet/task_100_worker_3.parquet
  5.5MB parquet/task_1010_worker_2.parquet


## Estrai pixel + Feature Engineering Sentinel-2

In [5]:
# ORDINE FINALE dopo stackstac.stack() (ALFABETICO!)
BANDS_ORDER = {
    0: 'blue',      # B02 (490nm)
    1: 'green',     # B03 (560nm)  
    2: 'nir',       # B08 (842nm) ← NIR principale
    3: 'nir08',     # B8A (865nm) ← NIR stretto
    4: 'red',       # B04 (665nm)
    5: 'rededge1',  # B05 (705nm)
    6: 'rededge2',  # B06 (740nm)
    7: 'rededge3',  # B07 (783nm)
    8: 'swir16',    # B11 (1610nm) ← SWIR1
    9: 'swir22'     # B12 (2190nm) ← SWIR2
}


In [6]:
# ✅ CORRETTO: Leggi PRIMO PARQUET
first_file = files[0]
print(f"🔍 Analizzo {first_file}")

obj = s3.get_object(Bucket='satellite-data', Key=first_file)
df_sample = pd.read_parquet(io.BytesIO(obj['Body'].read()))

print(f"📊 Shape: {df_sample.shape}")
print(f"📋 Colonne: {list(df_sample.columns)}")
print("\n📈 Prime 5 righe:")
print(df_sample.head())

# %%
# 🔬 IDENTIFICA BANDE (stackstac order alfabetico [file:1])
band_cols = [col for col in df_sample.columns if col.startswith('band_')]
print(f"\n🎯 {len(band_cols)} bande: {band_cols}")

# Statistiche bande
print("\n📊 STATISTICHE BANDE:")
for i, col in enumerate(band_cols):
    band_stats = df_sample[col].describe()
    print(f"  Band {i} ({col}): mean={band_stats['mean']:.1f} std={band_stats['std']:.1f} "
          f"range=[{band_stats['min']:.0f}, {band_stats['max']:.0f}]")

# %%
# 🌿 TEST NDVI sui parquet (Red=B04=band_4?, NIR=B08=band_2?)
print("\n🌿 TEST INDICI SPETTRALI:")

# Ordine stackstac alfabetico da worker.py [file:1]
# blue(0), green(1), nir(2), nir08(3), red(4), rededge1(5), rededge2(6), rededge3(7), swir16(8), swir22(9)
red_b4 = df_sample['band_4']    # B04 Red (index 4)
nir_b8 = df_sample['band_2']    # B08 NIR (index 2)  
swir_b11 = df_sample['band_8']  # B11 SWIR1 (index 8)
green_b3 = df_sample['band_1']  # B03 Green (index 1)

ndvi = (nir_b8 - red_b4) / (nir_b8 + red_b4 + 1e-8)
ndwi = (green_b3 - swir_b11) / (green_b3 + swir_b11 + 1e-8)

print(f"NDVI range: [{ndvi.min():.3f}, {ndvi.max():.3f}] (atteso: -0.2 → +0.8)")
print(f"NDWI range: [{ndwi.min():.3f}, {ndwi.max():.3f}] (acque <0)")
print(f"Crop ratio: {df_sample['label'].mean():.1%}")


🔍 Analizzo parquet/task_0_worker_9.parquet
📊 Shape: (257383, 13)
📋 Colonne: ['patch_id', 'pixel_idx', 'label', 'band_0', 'band_1', 'band_2', 'band_3', 'band_4', 'band_5', 'band_6', 'band_7', 'band_8', 'band_9']

📈 Prime 5 righe:
          patch_id  pixel_idx  label  band_0  band_1  band_2  band_3  band_4  \
0  task_0_worker_9          0      0    8976    8888    8904    9040    9160   
1  task_0_worker_9          1      0    8592    8552    8496    8416    8504   
2  task_0_worker_9          2      0    6216    5724    5508    5108    5512   
3  task_0_worker_9          3      0    6632    6352    5684    5180    5036   
4  task_0_worker_9          4      0    3408    3936    4492    4600    4552   

   band_5  band_6  band_7  band_8  band_9  
0    9152    8944    8744    8688    8616  
1    8504    8384    7920    7264    6708  
2    5824    6420    6720    6688    6612  
3    3932    2740    2610    2708    3030  
4    4480    4716    5156    5340    5380  

🎯 10 bande: ['band_0', 'b

## 📊 CELL 5: Estrai pixel → DataFrame 6 colonne


### STEP 1: Lista patch S3 (100 patch per test veloce)

In [8]:
from pyspark.sql.functions import *
pixels_df = spark.read.parquet("s3a://satellite-data/parquet/*.parquet")

print("📊 Dataset globale...")
total = pixels_df.count()
crop_count = pixels_df.filter(col("label") == 1).count()
print(f"Total: {total:,} | Crop: {crop_count:,} ({crop_count/total*100:.1f}%)")

# %%
# ✅ NORMALIZZAZIONE + FEATURES (valori / 10000.0)
features_df = pixels_df.select(
    (col("band_4") / 10000.0).alias("red_b4"),     # B04 → 0-1
    (col("band_2") / 10000.0).alias("nir_b8"),     # B08 → 0-1
    (col("band_8") / 10000.0).alias("swir_b11"),   # B11 → 0-1
    (col("band_1") / 10000.0).alias("green_b3"),   # B03 → 0-1
    col("label").cast("float")
)

# 🌿 INDICI SPETTRALI (sicuri)
features_df = features_df.withColumn("ndvi", 
    F.when((col("nir_b8") + col("red_b4")) > 0.001,  # ← Threshold anti-zero
           (col("nir_b8") - col("red_b4")) / (col("nir_b8") + col("red_b4")))
    .otherwise(0.0)) \
.withColumn("ndwi", 
    F.when((col("green_b3") + col("swir_b11")) > 0.001,
           (col("green_b3") - col("swir_b11")) / (col("green_b3") + col("swir_b11")))
    .otherwise(0.0)) \
.withColumn("ndmi", 
    F.when((col("nir_b8") + col("swir_b11")) > 0.001,
           (col("nir_b8") - col("swir_b11")) / (col("nir_b8") + col("swir_b11")))
    .otherwise(0.0)) \
.select("red_b4", "nir_b8", "swir_b11", "ndvi", "ndwi", "ndmi", "label") \
.filter(col("label").isin(0.0, 1.0)).repartition(200)

print("✅ Features normalizzate!")
features_df.describe().show()


25/12/29 14:53:12 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


📊 Dataset globale...


Total: 444,875,391 | Crop: 89,192,236 (20.0%)
✅ Features normalizzate!


25/12/29 14:53:45 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
ERROR:root:KeyboardInterrupt while sending command.==========>  (191 + 9) / 200]
Traceback (most recent call last):
  File "/home/amministratore/miniconda3/envs/spark_env/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/amministratore/miniconda3/envs/spark_env/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/amministratore/miniconda3/envs/spark_env/lib/python3.8/socket.py", line 681, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
del pixels_df
gc.collect()

## ML Pipeline + Training

In [ ]:
from pyspark.sql.functions import col, lit, rand

# 🎯 BILANCIA COMPLETO: Usa TUTTI i CROP + Sotto-campiona NON-CROP
print("Forzo bilanciamento 50/50 usando TUTTI i crop...")
crop_df = features_df.filter(col("label") == 1.0)   # CROP (usa TUTTI!)
noncrop_df = features_df.filter(col("label") == 0.0) # NON-CROP

print(f"📊 Originale: Crop={crop_df.count():,}, NonCrop={noncrop_df.count():,}")

# 🔄 Calcola quante NON-CROP prendere per matchare CROP
n_crop = crop_df.count()
n_noncrop_needed = n_crop  # Stesso numero di noncrop per 50/50

# SPLIT 80/20 per ENTRAMBI
train_crop, test_crop = crop_df.randomSplit([0.8, 0.2], seed=42)
train_noncrop = noncrop_df.sample(False, 0.8 * n_noncrop_needed / noncrop_df.count(), seed=42)
test_noncrop = noncrop_df.sample(False, 0.2 * n_noncrop_needed / noncrop_df.count(), seed=42)

train_df = train_crop.union(train_noncrop)
test_df = test_crop.union(test_noncrop)

print(f"✅ Train 50/50: Crop={train_df.filter(col('label')==1).count():,}, NonCrop={train_df.filter(col('label')==0).count():,}")
print(f"✅ Test 50/50: Crop={test_df.filter(col('label')==1).count():,}, NonCrop={test_df.filter(col('label')==0).count():,}")
print(f"📊 % usato: Crop=100%, NonCrop={((train_noncrop.count() + test_noncrop.count()) / noncrop_df.count() * 100):.1f}%")

In [ ]:
# Pipeline invariata (già perfetta)
assembler = VectorAssembler(
    inputCols=["red_b4", "nir_b8", "swir_b11", "ndvi", "ndwi", "ndmi"], 
    outputCol="features_vec",
    handleInvalid="skip"
)

scaler = StandardScaler(inputCol="features_vec", 
                        outputCol="scaled_features", 
                        withMean=True, 
                        withStd=True)

rf = RandomForestClassifier(
    featuresCol="scaled_features", labelCol="label",
    numTrees=100, maxDepth=10, impurity="gini",
    featureSubsetStrategy="sqrt", seed=42
)


pipeline = Pipeline(stages=[assembler, scaler, rf])

# 🚀 TRAIN UNA VOLTA (bilanciato al 100%!)
model = pipeline.fit(train_df)
print("✅ Modello addestrato 50/50!")

📊 Train: 0 | Test: 0 pixel


IllegalArgumentException: Data type array<float> of column features is not supported.

In [ ]:
# 📊 TEST
predictions = model.transform(test_df)

# Metriche
auc = BinaryClassificationEvaluator(labelCol="label").evaluate(predictions)
f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1").evaluate(predictions)

print(f"🎯 AUC-ROC: {auc:.4f}")
print(f"📈 F1-Score: {f1:.4f}")

# Confusion Matrix
predictions.groupBy("label", "prediction").count().orderBy("label", "prediction").show()

# 🌿 Feature Importance
importances = model.stages[-1].featureImportances
features = ["Red(B4)", "NIR(B8)", "SWIR(B11)", "NDVI", "NDWI", "NDMI"]
for i, (feat, imp) in enumerate(zip(features, importances)):
    print(f"🌿 {feat}: {imp:.3f}")


In [ ]:
model.write().overwrite().save("s3a://satellite-data/models/crop_rf_v1")
print("✅ SALVATO su MinIO!")


In [ ]:
spark.stop()
print("✅ Spark terminato!")